In [1]:
import numpy as np
import math
#import matplotlib.pyplot as plt
import tensorflow as tf 
import scipy.stats
import copy
import os

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def Data_Aggre(RawData,Time_interval,ave):
    # Time_interval表示一个时间间隔的长度（mins）
    AggStep = np.int(Time_interval/10)
    if RawData.shape[0]> RawData.shape[1]:
        Nm_inter = RawData.shape[1]
        Total_inter = RawData.shape[0]
        Time_intervalNum = np.int(Total_inter/AggStep)
        print("Time_intervalNum:",Time_intervalNum)
        AggregateData = np.zeros([Time_intervalNum,Nm_inter]) 
        for i in range(Time_intervalNum):
            for k in range(AggStep):
                AggregateData[i,:] = AggregateData[i,:]+ RawData[AggStep*i+k,:]        
    # 正常情况
    else:
        Nm_inter = RawData.shape[0]
        Total_inter = RawData.shape[1]
        Time_intervalNum = np.int(Total_inter/AggStep)
        print("Time_intervalNum:",Time_intervalNum)
        AggregateData = np.zeros([Nm_inter,Time_intervalNum]) 
        for i in range(0,Time_intervalNum):
            for k in range(0,AggStep):
                AggregateData[:,i] = AggregateData[:,i]+ RawData[:,AggStep*i+k]
    if ave==False:
        return AggregateData
    else: 
        return AggregateData/AggStep  #返回时间步中的平均速度

#Acc_30m = Data_Aggre(Acc,Time_interval,ave=False)

In [2]:
# 21744帧的情况
T_start = 9000
T_end = 20000

Total_Num = 21744
Acc_total = np.load('Dataset/Acc_sum.npy') #5 month 01~05/2017
Pickup_total = np.load('Dataset/Taxipick_sum.npy')
Dropoff_total = np.load('Dataset/Taxidrop_sum.npy')
Taxi_pickd_total = np.load('Dataset/Taxi_pick_diff.npy') 
Speed_total = np.load('Dataset/Speed_sum_fillFM.npy') 
Y_coarse = np.load('Dataset/Y_Coarse_Risk.npy')
External_total = np.load('Dataset/ex_time_stamp_10min.npy')
Static_affinity = np.load('Dataset/static_affinity.npy')


AccSUM =np.sum(Acc_total,axis=0)
Speed_total = np.where(Speed_total>100,0,Speed_total)
Speed_total = np.where(Speed_total>75,75,Speed_total)
Speed_total_diff = np.zeros([354, Total_Num])
# 选定哪些区域有taxi经过 可以计算交通模式的相似度
TaxiRegIndex = list(np.flatnonzero(np.sum(Pickup_total,axis = 1)))
for i in range(1,Total_Num):
    Speed_total_diff[:,i] = abs(Speed_total[:,i]- Speed_total[:,i-1])

    
# 对pickup的数据做处理（log化减小差距）
Taxi_pickd_total = np.log2(Taxi_pickd_total+2)
Pickup_total = np.log2(Pickup_total+2)
Speed_total = Speed_total/10
# 对speed进行处理
Mean_speed = np.mean(Speed_total)
Speed_total = np.where(Speed_total==0,Mean_speed,Speed_total)
Speed_total_diff = np.log2(Speed_total_diff+2)
Acc_prob_negnew = np.load('Dataset/Acc_prob_negnew.npy')
Acc_total_new = np.zeros([Acc_total.shape[0],Acc_total.shape[1]])
for i in range(354):
    Acc_total_new[i,:] = np.where(Acc_total[i,:] == 0, Acc_prob_negnew[i],Acc_total[i,:]*3.5)
Acc_total = Acc_total_new/10

In [ ]:
import scipy.stats
def JS_divergence(p,q):
    M=(p+q)/2
    return 0.5*scipy.stats.entropy(p, M)+0.5*scipy.stats.entropy(q, M)

In [ ]:
def Cal_Matrix(t):
    TrafficPatt = np.zeros([354,7])
    for i in range(7):
        TrafficPatt[:,i] = Pickup_total[:,t-144*i]
    Dynamic_Aff = np.zeros([354,354])
    for i in TaxiRegIndex:
        for j in TaxiRegIndex:
            if np.sum(TrafficPatt[i,:])!=0 and np.sum(TrafficPatt[j,:])!=0:
                Dynamic_Aff[i,j]= JS_divergence(TrafficPatt[i,:],TrafficPatt[j,:])
                Dynamic_Aff[i,j] = np.exp(-Dynamic_Aff[i,j]/0.35)
                Dynamic_Aff[j,i] = Dynamic_Aff[i,j]
    Dynamic_Aff = Dynamic_Aff+Static_affinity
    
#     A = Dynamic_Aff + np.eye(354) #Node_Num
#     D_12 = np.diag(np.power(np.array(A.sum(1)), -0.5).flatten(), 0)
#     L = A.dot(D_12).transpose().dot(D_12)*10
#    return L
    return Dynamic_Aff
    

In [ ]:
def Construct_inp_out(t,h): # t，h 表示对某一个时刻t之后h个interval进行预测，Acc_Risk, Pick, Speed, Pick_diff, Speed_diff
    # organize input sequence  
    scale = 10
    Acc_D1 = (np.mean(Acc_total[:,t-432:t-288],axis=1)*scale).reshape([354,1])
    Acc_D2 = ((Acc_total[:,t-432]+Acc_total[:,t-288])).reshape([354,1])/2
    Acc_R6 = Acc_total[:,t-5:t+1].reshape([354,6])
    input_seq_Acc = np.concatenate((Acc_D1,Acc_D2,Acc_R6),axis=1)

    Pick_D1 = (np.mean(Pickup_total[:,t-432:t-288],axis=1)).reshape([354,1])
    Pick_D2 = ((Pickup_total[:,t-432]+Pickup_total[:,t-288])).reshape([354,1])/2
    Pick_R6 = Pickup_total[:,t-5:t+1].reshape([354,6])
    input_seq_Pick = np.concatenate((Pick_D1,Pick_D2,Pick_R6),axis=1)

    Speed_D1 = (np.mean(Speed_total[:,t-432:t-288],axis=1)).reshape([354,1])
    Speed_D2 = ((Speed_total[:,t-432]+Speed_total[:,t-288])).reshape([354,1])/2
    Speed_R6 = Speed_total[:,t-5:t+1].reshape([354,6])
    input_seq_Speed = np.concatenate((Speed_D1,Speed_D2,Speed_R6),axis=1)

    Pickdiff_D1= (np.mean(Taxi_pickd_total[:,t-432:t-288],axis=1)).reshape([354,1])
    Pickdiff_D2 = ((Taxi_pickd_total[:,t-432]+Taxi_pickd_total[:,t-288])).reshape([354,1])/2
    Pickdiff_R6 = Taxi_pickd_total[:,t-5:t+1].reshape([354,6])
    input_seq_PickDif = np.concatenate((Pickdiff_D1,Pickdiff_D2,Pickdiff_R6),axis=1)


    Speeddf_D1= (np.mean(Speed_total_diff[:,t-432:t-288],axis=1)).reshape([354,1])
    Speeddf_D2 = ((Speed_total_diff[:,t-432]+Speed_total_diff[:,t-288])).reshape([354,1])/2
    Speeddf_R6 = Speed_total_diff[:,t-5:t+1].reshape([354,6])
    input_seq_SpeedDif = np.concatenate((Speeddf_D1,Speeddf_D2,Speeddf_R6),axis=1)
    
    Ycoarse_D1 = np.mean(Y_coarse[:,t-432:t-288],axis=1).reshape([1,18])
    Ycoarse_D2 = ((Y_coarse[:,t-432]+Y_coarse[:,t-288])/2).reshape([1,18])
    Ycoarse_R6 = Y_coarse[:,t-5:t+1].transpose()
    input_seq_Ycoarse = np.concatenate([Ycoarse_D1,Ycoarse_D2,Ycoarse_R6])
    
    
    Input_Seq = []
    Input_Seq.append(input_seq_Acc)
    Input_Seq.append(input_seq_Pick)
    Input_Seq.append(input_seq_Speed)
    Input_Seq.append(input_seq_PickDif)
    Input_Seq.append(input_seq_SpeedDif)
    Input_Seq = np.array(Input_Seq)


    External_D1 = np.mean(External_total[t-432:t-288,:],axis=0).reshape([1,9])
    External_D1[0,3] = 0
    External_D2 = ((External_total[t-432,:]+External_total[t-288,:])/2).reshape([1,9])
    External_R6 = External_total[t-5:t+1,:]
    input_Ext = np.concatenate((External_D1,External_D2,External_R6),axis=0)

    ##-------------option to hide these codes-------------#
    # Matrix_D1 = (Cal_Matrix(t-432)+Cal_Matrix(t-288))/2
    # Matrix_D1 = Matrix_D1.reshape([1,354,354])
    # Matrix_D2 = Matrix_D1
    # Matrix_D2 = Matrix_D2.reshape([1,354,354])
    # Matrix_R6 = np.zeros([354,354])
    # for i in range(0,6):
    #      Matrix_R6 = Matrix_R6 + Cal_Matrix(t-i*144)
    # Matrix_R6 = Matrix_R6/6
    # Matrix_R6 = Matrix_R6.reshape([1,354,354])
    #Matrix_Aff = np.concatenate((Matrix_D1,Matrix_D2,Matrix_R6),axis=0)
    ##-------------option to hide these codes-------------#

    ##-------------option to hide these codes-------------#
    Matrix_D1 = Cal_Matrix(t-3).reshape([1,354,354])
    Matrix_Aff = np.concatenate((Matrix_D1,Matrix_D1,Matrix_D1),axis=0)
    ##-------------option to hide these codes-------------#
    # organize output sequence
    Output_Risk = Acc_total[:,t+1:t+h+1].transpose()
    Output_Ycoarse = Y_coarse[:,t+1:t+h+1].transpose()
    Output_Ext = External_total[t+1:t+h+1,:]
    Output_AccSUM = AccSUM[t+1:t+h+1]
    return Input_Seq, input_Ext, Matrix_Aff, Output_Risk, input_seq_Ycoarse, Output_Ycoarse,Output_Ext,Output_AccSUM

In [ ]:
def generate_samples(t1,t2,batch_size):
    tt = np.random.choice(range(t1,t2), batch_size)
#     tt = np.array([10287, 10289, 10293, 10296, 10298, 10302,
#        10304, 10305, 10307, 10309, 10311, 10313, 10315, 10316, 10318,
#        10320, 10322, 10323, 10325, 10327, 10329, 10330, 10332, 10334,
#        10336, 10338, 10341, 10342, 10343, 10345])
    Input_Batch = []
    Intput_Ycoarse_B = []
    Input_Ext_B = []
    Matrix_Aff_B = []
    Output_Risk_B = []
    Output_Ycoarse_B = []
    Output_AccSUM_B = []
    Output_Ext_B = []
    for i in tt:
        Input_Seq, input_Ext, Matrix_Aff, Output_Risk, input_seq_Ycoarse, Output_Ycoarse,Output_Ext,Output_AccSUM = Construct_inp_out(i,6)
        Input_Batch.append(Input_Seq)
        Input_Ext_B.append(input_Ext)
        Matrix_Aff_B.append(Matrix_Aff)
        Output_Risk_B.append(Output_Risk)
        Output_Ycoarse_B.append(Output_Ycoarse)
        Output_Ext_B.append(Output_Ext)
        Intput_Ycoarse_B.append(input_seq_Ycoarse)
        Output_AccSUM_B.append(Output_AccSUM)
    Input_Batch = np.array(Input_Batch)
    Input_Ext_B = np.array(Input_Ext_B)
    Matrix_Aff_B = np.array(Matrix_Aff_B)
    Output_Risk_B = np.array(Output_Risk_B)
    Intput_Ycoarse_B = np.array(Intput_Ycoarse_B)
    Output_Ycoarse_B = np.array(Output_Ycoarse_B)
    Output_Ext_B = np.array(Output_Ext_B)
    Output_AccSUM_B = np.array(Output_AccSUM_B)
    return Input_Batch,Input_Ext_B,Matrix_Aff_B,Output_Risk_B,Intput_Ycoarse_B,Output_Ycoarse_B,Output_Ext_B,Output_AccSUM_B,tt

In [ ]:
def build_graph(N_Ad,SL_Ad,Gra_Adj,HF_A,HC_A, T1,T2,T3,feed_previous = False,reuse_variables=False):
    tf.reset_default_graph()

    N_num_Adj = [64,128,192,256,320,512]  #6  default = 1 N_Ad = 3, SL_Ad = 1,Gra_Adj = 2, HF_A = 2,HC_A =2,T1 = 2, T2 = 2, T3 = 2
    num_stacked_layers_Adj = [1,2,3,4]    #4  default = 1 SL_Ad = 1
    GRADIENT_CLIPPING_Adj = [1.5,2.0,2.5,3.0] #4  default = 2 Gra_Adj = 2
    H_F_Adj = [128,192,256,320]           #4  default = 2 HF_A = 2 
    H_C_Adj = [9,16,32,48,64]                #4  default = 2 HC_A =2
    Lamda_Adj = [0.5,0.8,1,1.2,1.5]       #5 default = 2, #default = 2,#default = 2  T1 = 2, T2 = 2, T3 = 2
    Layer_GCN = 6
    N_num = N_num_Adj[N_Ad]
    Ex_num = 9
    Attribute_Num = 5
    Y_coarse_dim = 18
     ## Network Parameters
    # length of input signals
    input_seq_len = 8 
    # length of output signals
    output_seq_len = 6 
    # num of stacked lstm layers 
    num_stacked_layers = num_stacked_layers_Adj[SL_Ad]
    # gradient clipping - to avoid gradient exploding
    GRADIENT_CLIPPING = GRADIENT_CLIPPING_Adj[Gra_Adj] 
    # input size 表示输入序列的长度，比如输入序列长度是20，输出12，就是利用前20个连续序列来预测第21-32个值 
    # size of LSTM Cell
    hidden_dim_F = H_F_Adj[HF_A]
    hidden_dim_C = H_C_Adj[HC_A]
    # num of input signals
    input_dim_F = 354
    input_dim_C = 18
    # num of output signals
    output_dim_F = 354
    output_dim_C = 18
    # Multi-task weight
    lamda_1 = Lamda_Adj[T1]
    lamda_2 = Lamda_Adj[T2]
    lamda_3 = Lamda_Adj[T3]
    # Learning_rate
    learning_rate = 0.001
    # Regularization rate
    lambda_l2_reg = 0.0001 
    
    #with tf.variable_scope('Seq2seq'):
        # Encoder: inputs
    def weight_variable(shape):
        initial=tf.truncated_normal(shape,stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial=tf.truncated_normal(shape,stddev=0.1)
        return tf.Variable(initial)

    #with tf.variable_scope('GCN'):
    #-----------输入input———————————————#
    # 5 (Attribute)    //      8 interval = 2 (expected) + 6(continous)
    # Acc_Risk, Pick, Speed, Pick_diff, Speed_diff
    Input_Seq = tf.placeholder(shape=(None, Attribute_Num, 354, input_seq_len),dtype=tf.float32) 
    Input_Ext = tf.placeholder(shape=(None, input_seq_len, Ex_num),dtype=tf.float32)
    Matrix_Aff = tf.placeholder(shape=(None, 3, 354, 354),dtype=tf.float32)
    Output_Seq = tf.placeholder(shape=(None, output_seq_len, 354),dtype=tf.float32)
    Ext_O = tf.placeholder(shape=(None, output_seq_len, Ex_num),dtype=tf.float32)
    Intput_Ycoarse = tf.placeholder(shape=(None, input_seq_len, Y_coarse_dim),dtype=tf.float32)
    Output_Ycoarse = tf.placeholder(shape=(None, output_seq_len, Y_coarse_dim),dtype=tf.float32)
    Acc_SumS = tf.placeholder(shape=(None, output_seq_len),dtype=tf.float32)
    Acc_Risk = Input_Seq[:,0,:,:]
    Pick = Input_Seq[:,1,:,:]
    Speed = Input_Seq[:,2,:,:]
    Pick_diff = Input_Seq[:,3,:,:]
    Speed_diff = Input_Seq[:,4,:,:]
    Dynamic_Feature = []
    for i in range(8):
        Input = tf.concat([tf.expand_dims(Acc_Risk[:,:,i],-1),tf.expand_dims(Pick[:,:,i],-1),tf.expand_dims(Speed[:,:,i],-1),tf.expand_dims(Pick_diff[:,:,i],-1),tf.expand_dims(Speed_diff[:,:,i],-1)],axis=-1)
        Dynamic_Feature.append(Input)
    Dynamic_Feature = tf.convert_to_tensor(Dynamic_Feature)
    def gen_gcn_weights():
        gcn_weights = {}


        gcn_weights['gcn1'] = tf.Variable(tf.truncated_normal(shape=(Attribute_Num, N_num), mean=0, stddev=0.1))#
        gcn_weights['gcn2'] = tf.Variable(tf.truncated_normal(shape=(N_num, N_num), mean=0, stddev=0.1))#
        gcn_weights['gcn9'] = tf.Variable(tf.truncated_normal(shape=(N_num, 1), mean=0, stddev=0.1))#
        gcn_weights['fusionH'] = tf.Variable(tf.truncated_normal(shape=(output_dim_C + Ex_num, output_dim_C)))#
        gcn_weights['fusion2'] = tf.Variable(tf.truncated_normal(shape=(2, 1), mean=0, stddev=0.1))#
        gcn_weights['fusion3'] = tf.Variable(tf.truncated_normal(shape=(64, 32), mean=0, stddev=0.1))#
        gcn_weights['fusion3'] = tf.Variable(tf.truncated_normal(shape=(64, 32), mean=0, stddev=0.1))#
        gcn_weights['embeds'] = tf.Variable(tf.truncated_normal(shape=(6, 6), mean=0, stddev=0.1))
        gcn_weights['high_low'] = tf.Variable(tf.truncated_normal(shape=(256, 128), mean=0, stddev=0.1))
        gcn_weights['Seq_out_WF'] = tf.Variable(tf.truncated_normal(shape=(hidden_dim_F, output_dim_F), dtype = tf.float32, mean=0, stddev=0.1))#
        gcn_weights['Seq_out_bF'] = tf.Variable(tf.truncated_normal(shape=(output_dim_F,), dtype = tf.float32, mean=0, stddev=0.1))#,mean=0, stddev=0.1

        gcn_weights['Seq_out_WC'] = tf.Variable(tf.random_normal(shape=(hidden_dim_C, output_dim_C), dtype = tf.float32))
        gcn_weights['Seq_out_bC'] = tf.Variable(tf.random_normal(shape=(output_dim_C,), dtype = tf.float32))

      #  gcn_weights['Gated_W'] = tf.Variable(tf.truncated_normal(shape=(output_dim,), dtype = tf.float32,mean=0, stddev=0.1))
        gcn_weights['Ext_W'] = tf.Variable(tf.truncated_normal(shape=(input_dim_C+Ex_num,output_dim_C), dtype = tf.float32,mean=0, stddev=0.1))#,mean=0, stddev=0.1
        gcn_weights['Ext_W_O'] = tf.Variable(tf.truncated_normal(shape=(Ex_num,input_dim_C), dtype = tf.float32,mean=0, stddev=0.1))#,mean=0, stddev=0.1
        gcn_weights['Coarse2Fine'] = tf.Variable(tf.truncated_normal(shape=(Y_coarse_dim,354), dtype = tf.float32, mean=0, stddev=0.1))#,mean=0, stddev=0.1
        gcn_weights['Fine2Coarse'] = tf.Variable(tf.truncated_normal(shape=(354,Y_coarse_dim), dtype = tf.float32))
        gcn_weights['Coarse2Sum_W'] = tf.Variable(tf.random_normal(shape=(Y_coarse_dim,1), dtype = tf.float32))
        gcn_weights['Coarse2Sum_b'] = tf.Variable(tf.random_normal(shape=(1,), dtype = tf.float32))        
        return gcn_weights
    weights=gen_gcn_weights()

    def GCN_1(input1,input_external,ex_num,Adj_M):
        #embedding
        # input1 = tf.map_fn(lambda x: tf.nn.leaky_relu(tf.matmul(x,weights['embeds']),alpha=0.8,name=None), input1)
        #定义第一层网络结构 GCN
        #layer1_temp = tf.map_fn(lambda x: tf.matmul(Adj_M, x), input1) #对于input1中的每个部分 都和A_tensor左乘
        layer1_temp = tf.matmul(Adj_M, input1)
        layer_1_output = tf.map_fn(lambda x: tf.matmul(x, weights['gcn1']), layer1_temp)
        # print(layer_1_output.shape)
        #定义第二层网络结构 GCN
        #layer2_temp = tf.map_fn(lambda x: tf.matmul(Adj_M, x), layer_1_output)
        layer2_temp = tf.matmul(Adj_M, layer_1_output)
        layer_2_output = tf.map_fn(lambda x: tf.matmul(x, weights['gcn2']), layer2_temp)
        #  layer_2_output=tf.nn.elu(layer_2_output)
        #layer_con2_5 = tf.map_fn(lambda x: tf.matmul(x, weights['fusion3']), layer_2_output)
        
        layer_2_output=tf.nn.leaky_relu(layer_2_output,alpha=0.8, name=None)    
        
        
        img_shape = [4,354,64]
        #Wx_plus_b = tf.Variable(tf.random_normal(img_shape))
        axis = list(range(len(img_shape) - 1))
        wb_mean, wb_var = tf.nn.moments(layer_2_output, [0,1])
        scale = tf.Variable(tf.ones([N_num]))
        offset = tf.Variable(tf.zeros([N_num]))
        variance_epsilon = 0.001
        layer_2_output = tf.nn.batch_normalization(layer_2_output, wb_mean, wb_var, offset, scale, variance_epsilon)
        
        
        #定义第三层网络结构 GCN
        #layer3_temp = tf.map_fn(lambda x: tf.matmul(Adj_M, x), layer_2_output)
        layer3_temp = tf.matmul(Adj_M, layer_2_output)
        layer_3_output = tf.map_fn(lambda x: tf.matmul(x, weights['gcn2']), layer3_temp)


        # layer_3_output=tf.nn.leaky_relu(layer_3_output,alpha=0.8, name=None)
        #定义第四层网络结构 GCN
        #layer4_temp = tf.map_fn(lambda x: tf.matmul(Adj_M, x), layer_3_output)
        layer4_temp = tf.matmul(Adj_M, layer_3_output)
        layer_4_output = tf.map_fn(lambda x: tf.matmul(x, weights['gcn2']), layer4_temp)      
        
        layer_4_output = tf.nn.leaky_relu(layer_4_output,alpha=0.8, name=None)
        
        
        wb_mean, wb_var = tf.nn.moments(layer_4_output, [0,1])
        scale = tf.Variable(tf.ones([N_num]))
        offset = tf.Variable(tf.zeros([N_num]))
        layer_4_output = tf.nn.batch_normalization(layer_4_output, wb_mean, wb_var, offset, scale, variance_epsilon)
        
        # 先做BN再做Leaky_ReLU
        
        layer5_temp = tf.matmul(Adj_M, layer_4_output)
        layer_5_output = tf.map_fn(lambda x: tf.matmul(x, weights['gcn2']), layer5_temp)
        
        Layer5ComBine = tf.add(layer_2_output,layer_4_output)  
        Layer5ComBine = tf.add(Layer5ComBine,layer_5_output) 

        
        layer6_temp = tf.matmul(Adj_M, Layer5ComBine)
        layer_6_output = tf.map_fn(lambda x: tf.matmul(x, weights['gcn9']), layer6_temp)
        
        wb_mean, wb_var = tf.nn.moments(layer_6_output, [0,1])
        scale = tf.Variable(tf.ones([1]))
        offset = tf.Variable(tf.zeros([1]))
        layer_6_output = tf.nn.batch_normalization(layer_6_output, wb_mean, wb_var, offset, scale, variance_epsilon)
        layer_6_output = tf.nn.leaky_relu(layer_6_output,alpha=0.8, name=None)       
        
        
        output= tf.squeeze(layer_6_output , -1) 
        keep_prob=0.5
        External_W = weight_variable([Ex_num,354])  
        External_b = bias_variable([354])
        External_out = tf.nn.leaky_relu(tf.matmul(input_external,External_W)+External_b,alpha=0.8, name=None)
       # External_out = tf.matmul(input_external,External_W)+External_b
        output=output + External_out

        return output

    #----------------------------Risk-Guided LSTMs -----(Seq2Seq Model)--------------------------#
    # 细粒度风险feature map的输入序列和输出序列 （F）
    Input_Seq_F = []
    for i in range(8):
        if i<=1:
            Input_Seq_F.append(GCN_1(Dynamic_Feature[i],Input_Ext[:,i],9,Matrix_Aff[:,i]))
        if i>=2:
            Input_Seq_F.append(GCN_1(Dynamic_Feature[i],Input_Ext[:,i],9,Matrix_Aff[:,2])) 

    Output_Seq_F = []  # 6*354
    for i in range(6):
        Output_Seq_F.append(Output_Seq[:,i,:])            


    # 粗粒度风险feature map的输入和输出（C）
    Input_Seq_C = []
    for i in range(8):
        Input_Seq_C.append(Intput_Ycoarse[:,i,:])
    Output_Seq_C = [] # 6*18
    for i in range(6):
        Output_Seq_C.append(Output_Ycoarse[:,i,:])


    # Decoder: target outputs  
    # 细粒度事故风险目标输出  
    target_seq_F = [
        tf.placeholder(tf.float32, shape=(None, output_dim_F), name="y".format(t))
          for t in range(output_seq_len)
    ]

    #  粗粒度事故风险目标输出
    target_seq_C = [
        tf.placeholder(tf.float32, shape=(None, output_dim_C), name="y".format(t))
          for t in range(output_seq_len)
    ]
    target_seq_F = Output_Seq_F
    target_seq_C = Output_Seq_C


    # Give a "GO" token to the decoder. 
    # If dec_inp are fed into decoder as inputs, this is 'guided' training; otherwise only the 
    # first element will be fed as decoder input which is then 'un-guided'
    # 这里有个target_seq的shape
    # 细粒度的decoder input
    dec_inp_F = [ tf.zeros_like(target_seq_F[0], dtype=tf.float32, name="GO") ] + target_seq_F[:-1]

    # 粗粒度的decoder input  加context factors
    dec_inp_C = [ tf.zeros_like(target_seq_C[0], dtype=tf.float32, name="GO") ] + target_seq_C[:-1]
    print("dec_inp_C:",dec_inp_C)

    dec_inp_extC = [tf.matmul(Ext_O[:,k,:],weights['Ext_W_O']) for k in range(output_seq_len)]
    dec_inp_extC = [ _i+_j  for _i,_j in zip(dec_inp_extC,dec_inp_C)]
#     dec_inp_extC = [tf.concat([Ext_O[:,k,:],dec_inp_C[k]],axis = -1) for k in range(output_seq_len)]
#     dec_inp_extC = [tf.matmul(dec_inp_extC[k], weights['fusionH']) for k in range(output_seq_len)] 
#    dec_inp_extC = [ _i+_j  for _i,_j in zip(dec_inp_extC,dec_inp_C)]
    print("dec_inp_extC:",dec_inp_extC)

    # 得到Accident的总数
    Acc_SumOut = []
    for i in range(6):
        Acc_SumOut.append(Acc_SumS[:,i])


    # -------------------此处可以加新的context_factors到dec_inp: concatenate----------------------------#


    def _rnn_decoder(decoder_inputs,
                    initial_state,
                    cell,
                    loop_function=None,
                    scope=None):

        state = initial_state
        outputs = []
        prev = None
        for i, inp in enumerate(decoder_inputs):
          if loop_function is not None and prev is not None:
            with tf.variable_scope("loop_function", reuse=True):
              inp = loop_function(prev, i)
          if i > 0:
            tf.get_variable_scope().reuse_variables()
          output, state = cell(inp, state)
          outputs.append(output)
          if loop_function is not None:
            prev = output
        return outputs, state

    def _loop_function(prev, _):
        #print(prev)
        if prev.shape[1] == hidden_dim_C:
            return tf.nn.relu(tf.matmul(prev, weights['Seq_out_WC']) + weights['Seq_out_bC'])
        if prev.shape[1] == hidden_dim_F:
            return tf.nn.relu(tf.matmul(prev, weights['Seq_out_WF']) + weights['Seq_out_bF'])

    def _basic_rnn_seq2seq(encoder_inputs,
                      decoder_inputs,
                      cell,
                      feed_previous,
                      dtype=tf.float32,
                      scope=None):
        # Encoder  
        enc_cell = cell
        _, enc_state = tf.contrib.rnn.static_rnn(enc_cell, encoder_inputs, dtype=dtype)
        # Decoder 
        if feed_previous:
            return _rnn_decoder(decoder_inputs, enc_state, cell, _loop_function)
        else:
            return _rnn_decoder(decoder_inputs, enc_state, cell)



    def _rnn_decoder(decoder_inputs,
                        initial_state,
                        cell,
                        loop_function=None,
                        scope=None):

            state = initial_state
            outputs = []
            prev = None
            for i, inp in enumerate(decoder_inputs):
              if loop_function is not None and prev is not None:
                with tf.variable_scope("loop_function", reuse=True):
                  inp = loop_function(prev, i)
              if i > 0:
                tf.get_variable_scope().reuse_variables()
              output, state = cell(inp, state)
              outputs.append(output)
              if loop_function is not None:
                prev = output
            return outputs, state



    def _basic_rnn_seq2seq2(encoder_inputs,
                      decoder_inputs,
                      cell,
                      feed_previous,
                      dtype=tf.float32,
                      scope=None):
        # Encoder  
        enc_cell = cell
        _, enc_state = tf.contrib.rnn.static_rnn(enc_cell, encoder_inputs, dtype=dtype)
        # Decoder 
        if feed_previous:
            return _rnn_decoder(decoder_inputs, enc_state, cell, _loop_function)
        else:
            return _rnn_decoder(decoder_inputs, enc_state, cell)



    ## 设置LSTM的cell
    with tf.variable_scope("Fine_RNN"):  
        cells_F = []
        for i in range(num_stacked_layers):
            with tf.variable_scope('RNN_{}'.format(i)):
                cells_F.append(tf.contrib.rnn.LSTMCell(num_units = hidden_dim_F, activation = tf.nn.leaky_relu)) # activation = tf.nn.leaky_relu
        cell_F = tf.contrib.rnn.MultiRNNCell(cells_F)
        ## 把粗粒度和细粒度分别喂到LSTM中
    #
        dec_outputs_F, dec_memory_F = _basic_rnn_seq2seq(
        Input_Seq_F, 
        dec_inp_F,  ##输入带有context的数据
        cell_F, 
        feed_previous = feed_previous
        )   
    with tf.variable_scope("Coarse_RNN"):  
        cells_C = []
        for i in range(num_stacked_layers):
            with tf.variable_scope('RNN_{}'.format(i)):
                cells_C.append(tf.contrib.rnn.LSTMCell(num_units = hidden_dim_C, activation = tf.nn.relu)) # activation = tf.nn.leaky_relu
        cell_C = tf.contrib.rnn.MultiRNNCell(cells_C)
        print("111")
        dec_outputs_C, dec_memory_C = _basic_rnn_seq2seq(
        Input_Seq_C, 
        dec_inp_extC,  ##输入带有context的数据
        cell_C, 
        feed_previous = feed_previous
        )
    print("target_seq_C",target_seq_C[:-1])

    #print("basic:",tf.get_variable_scope().reuse)# basic_rnn_seq2seq会将reuse改变成resuse=True


    reshaped_outputsF = [tf.nn.leaky_relu(tf.matmul(i, weights['Seq_out_WF']) + weights['Seq_out_bF'],alpha=0.8) for i in dec_outputs_F]

    reshaped_outputsC = [tf.nn.relu(tf.matmul(i, weights['Seq_out_WC'])+ weights['Seq_out_bC']) for i in dec_outputs_C]

    reshaped_outputsF = [tf.nn.leaky_relu(tf.matmul(i, weights['Coarse2Fine']) + j,alpha=0.8) for i,j in zip(reshaped_outputsC,reshaped_outputsF)]


    SUM_outputs = [tf.nn.relu(tf.matmul(i, weights['Coarse2Sum_W']) + weights['Coarse2Sum_b']) for i in reshaped_outputsC]


    output_loss = 0
    output_lossT = 0
    output_lossC = 0
    output_lossF = 0
    reg_loss = 0

    for _y, _Y in zip(SUM_outputs, Acc_SumOut):
        output_lossT += tf.reduce_mean(tf.pow(_y - _Y, 2))

    for _y, _Y in zip(reshaped_outputsC, Output_Seq_C):
        output_lossC += tf.reduce_mean(tf.pow(_y - _Y, 2))

    for _y, _Y in zip(reshaped_outputsF, Output_Seq_F):
        output_lossF += tf.reduce_mean(tf.pow(_y - _Y, 2))

    for tf_var in tf.trainable_variables():
        if 'Seq_out_' in tf_var.name in tf_var.name:
            reg_loss += tf.reduce_mean(tf.nn.l2_loss(tf_var))
    # L2 regularization for weights and biases

    #  reg = tf.contrifb.layers.apply_regularization(tf.contrib.layers.l2_regularizer(1e-4),tf.trainable_variables())

    loss = lamda_1 * output_lossT + lamda_2 * output_lossC + lamda_3 * output_lossF + lambda_l2_reg * reg_loss
    global_step = tf.Variable(
              initial_value=0,
             # name="global_step",
              trainable=False,
              collections=[tf.GraphKeys.GLOBAL_STEP, tf.GraphKeys.GLOBAL_VARIABLES])
    #print(tf.get_variable_scope().reuse)
    starter_learning_rate = learning_rate
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 10, 0.75, staircase=True)
    with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
    #         optimizer = tf.contrib.layers.optimize_loss(
    #             loss=loss,
    #             learning_rate=learning_rate,
    #             global_step=global_step,
    #             optimizer='Adam',
    #             clip_gradients=GRADIENT_CLIPPING)
        # global_step=global_step,
        
        optimizer = tf.contrib.layers.optimize_loss(
        loss=loss,
        learning_rate = learning_rate,
        global_step=global_step,
        optimizer='Adam',
        clip_gradients=GRADIENT_CLIPPING)

    saver = tf.train.Saver
    print(SUM_outputs)
    Hyper_param = dict(
        Layer_GCN = Layer_GCN,
        N_num = N_num,
        Ex_num = Ex_num,
        # num of stacked lstm layers 
        num_stacked_layers = num_stacked_layers,
        # gradient clipping - to avoid gradient exploding
        GRADIENT_CLIPPING = GRADIENT_CLIPPING,
        # size of LSTM Cell
        hidden_dim_F = hidden_dim_F,
        hidden_dim_C = hidden_dim_C,
        lambda_l2_reg = lambda_l2_reg,
        learning_rate = learning_rate,
        lamda_1 = lamda_1,
        lamda_2 = lamda_2,
        lamda_3 = lamda_3
        
    )
    return dict(
        Input_Seq = Input_Seq,
        Output_Seq = Output_Seq,
        Input_Ext = Input_Ext, 
        Matrix_Aff = Matrix_Aff,
        Ext_O = Ext_O,
        Output_Ycoarse = Output_Ycoarse,
        train_op = optimizer, 
        loss=loss,
        saver = saver, 
        Acc_SumS = Acc_SumS,
        reshaped_outputsF = reshaped_outputsF,
        reshaped_outputsC = reshaped_outputsC,
        Intput_Ycoarse = Intput_Ycoarse,
        SUM_outputs = SUM_outputs,
        Hyper_param = Hyper_param
        )



In [ ]:
rnn_model = build_graph(N_Ad = 4, SL_Ad = 1,Gra_Adj = 2, HF_A = 3,HC_A =2,T1 = 3, T2 = 1, T3 = 2,feed_previous=False)
#     N_num_Adj = [64,128,192,256,320,512]  #6  default = 3 N_Ad = 3, SL_Ad = 1,Gra_Adj = 2, HF_A = 2,HC_A =2,T1 = 2, T2 = 2, T3 = 2
#     num_stacked_layers_Adj = [1,2,3,4]    #4  default = 1 SL_Ad = 1
#     GRADIENT_CLIPPING_Adj = [1.5,2,2.5,3] #4  default = 2 Gra_Adj = 2
#     H_F_Adj = [128,192,256,320]           #4  default = 2 HF_A = 2 
#     H_C_Adj = [9,16,32,64]                #4  default = 2 HC_A =2
#     Lamda_Adj = [0.5,0.8,1,1.2,1.5] (TCF) #5 default = 2, #default = 2,#default = 2  T1 = 2, T2 = 2, T3 = 2

# Training Process

In [ ]:
# 直接进行训练，需训练到35个epoch之后能达到50\%的命中率
import csv
#import datetime
#import evaluation
#rnn_model = build_graph(N_Ad = 3, SL_Ad = 1,Gra_Adj = 2, HF_A = 2,HC_A =2,T1 = 2, T2 = 2, T3 = 2,feed_previous=False)
saver = tf.train.Saver()
epoch_loss = []
# starttime = datetime.datetime.now()
init = tf.global_variables_initializer()
K = 30
epoch =100
Batchsize = 30
Output_seq_len = 6
Total_sample = 10000-9000

# print(Total_sample,Batchsize,Batchsize*2,Batch_Num)
with tf.Session() as sess:
    sess.run(init)
    max_Acc = 0
    Total_loss = 0
    Total_Accarcy = 0
    for i in range(epoch): 
        sub_loss_sum = 0
        final_preds_F = []
        Acc_Real = np.zeros([Output_seq_len])
        Acc_Iden = np.zeros([Output_seq_len])
        Input_Batch,Input_Ext_B,Matrix_Aff_B,Output_Risk_B,Input_Ycoarse_B, Output_Ycoarse_B,Output_Ext_B,SUMOut,SelectedTime = generate_samples(2500,4500,Batchsize)
        feed_dict = {rnn_model['Input_Seq']:Input_Batch, rnn_model['Input_Ext']:Input_Ext_B ,rnn_model['Matrix_Aff']:Matrix_Aff_B,rnn_model['Output_Seq']:Output_Risk_B,
                    rnn_model['Intput_Ycoarse']:Input_Ycoarse_B, rnn_model['Ext_O']:Output_Ext_B, rnn_model['Output_Ycoarse']: Output_Ycoarse_B,rnn_model['Acc_SumS']:SUMOut}
        _, loss_t,final_preds_f = sess.run([rnn_model['train_op'], rnn_model['loss'],rnn_model['reshaped_outputsF']], feed_dict)
        print(i,loss_t)
        final_preds_F.append(final_preds_f)
        final_predsF = np.array(final_preds_F)  
        Total_Acc = 0
        Pred_Acc = 0
        for batch in range(Batchsize):
            for seq_step in range(Output_seq_len):
                Real = list(np.flatnonzero((Output_Risk_B[batch,seq_step,:])>0)) # 10,6,354 batch_size,time_step,output_dim
                Predicted = list(np.argsort(-final_predsF[0,seq_step,batch,:])[0:K]) # 1,6,10,354  1,time_step,batch_size,output_dim
                Total_Acc = Total_Acc + len(Real)        
                Cross = list(set(Predicted).intersection(set(Real)))
                # print(Cross)
                Pred_Acc = Pred_Acc + len(Cross)
                Acc_Real[seq_step] = Acc_Real[seq_step] + len(Real)
                Acc_Iden[seq_step] = Acc_Iden[seq_step] + len(Cross)
        Total_Accarcy = Pred_Acc/Total_Acc
        print(Total_Acc,Pred_Acc,Pred_Acc/Total_Acc,Total_loss)
       
        if (Total_Accarcy> 0.5) & (np.flatnonzero(np.isnan(Acc_Real)).shape[0]==0): 
            print(Acc_Iden/Acc_Real)

        if Total_Accarcy> max_Acc:
            max_Acc = Pred_Acc/Total_Acc
            temp_saver = rnn_model['saver']()
    temp_saver = rnn_model['saver']()
    txt_file = "Hyper_param.txt"
    with open(txt_file,"a+", newline='') as ofile:
        field_names = ['Layer_GCN', 'N_num', 'Ex_num','num_stacked_layers','GRADIENT_CLIPPING', 'hidden_dim_F', 'hidden_dim_C', 'lambda_l2_reg',
        'learning_rate','lamda_1','lamda_2', 'lamda_3','Acc']

        odict = csv.DictWriter(ofile, field_names)
        row = rnn_model['Hyper_param']
        row2 = dict(Acc = max_Acc)
        odict.writerow(row)
        odict.writerow(row2)
        ofile.close()    
    save_path = temp_saver.save(sess, os.path.join('./', 'Multi_scaleFore_Results/CGcon_3_1_2_2_MixHop/univariate_testcitywide'))

print("Checkpoint saved at: ", save_path)

# Test

In [ ]:
# 测试集进行测试
Batchsize = 30
Output_seq_len = 6
rnn_model = build_graph(N_Ad = 4, SL_Ad = 1,Gra_Adj = 2, HF_A = 3,HC_A =2,T1 = 3, T2 = 1, T3 = 2,feed_previous=False)
init = tf.global_variables_initializer()
t1 = 9000
t2 = 20000

with tf.Session() as sess:
    final_preds_F = []
    final_preds_C = []
    Sum_Pred = []
    sess.run(init)
    saver = rnn_model['saver']().restore(sess, os.path.join('./', 'Multi_scaleFore_Results/CGcon_3_1_2_2_MixHop/univariate_testcitywide'))
    Test_Input_Batch,Test_Input_Ext_B,Test_Matrix_Aff_B,Test_Output_Risk_B,Test_Input_Ycoarse_B, Test_Output_Ycoarse_B,Test_Output_Ext_B,Test_SUM,SelectedTime = generate_samples(T_start,T_end,Batchsize)
    feed_dict = {rnn_model['Input_Seq']: Test_Input_Batch,rnn_model['Input_Ext']:Test_Input_Ext_B,rnn_model['Matrix_Aff']:Test_Matrix_Aff_B,rnn_model['Output_Seq']:Test_Output_Risk_B,
                rnn_model['Intput_Ycoarse']:Test_Input_Ycoarse_B, rnn_model['Ext_O']:Test_Output_Ext_B, rnn_model['Output_Ycoarse']: Test_Output_Ycoarse_B,rnn_model['Acc_SumS']:Test_SUM
                }          
    final_preds_f,final_preds_c,sum_outputs = sess.run([rnn_model['reshaped_outputsF'],rnn_model['reshaped_outputsC'],rnn_model['SUM_outputs']],feed_dict)
    final_preds_F.append(final_preds_f)
    final_preds_C.append(final_preds_c)
    Sum_Pred.append(sum_outputs)
final_preds_F = np.array(final_preds_F)
final_preds_C = np.array(final_preds_C)
Sum_Pred = np.array(Sum_Pred)


In [ ]:
# Testing process
K = 20
Total_Acc = 0
Pred_Acc = 0
Acc_Iden = np.zeros([6])
Acc_Real = np.zeros([6])
FError = 0
CError = 0
FMAPE = 0
CMAPE = 0
for batch in range(Batchsize):
    for seq_step in range(Output_seq_len):
        #batch = 5
        #seq_step = 0
        Real = list(np.flatnonzero((Test_Output_Risk_B[batch,seq_step,:])>0)) # 10,6,354 batch_size,time_step,output_dim
        # K = np.int(np.sum(final_preds_C[0,seq_step,batch,:]))
        K = 20
        # K = np.int(np.round(Sum_Pred[0,seq_step,batch,0]))+ 8 #Sum_Pred[0,seq_step,batch,0]
        # Predicted = Risk_Proposal(final_preds_F[0,seq_step,batch,:],final_preds_C[0,seq_step,batch,:])
        Predicted = list(np.argsort(-final_preds_F[0,seq_step,batch,:])[0:K]) # 1,6,10,354  1,time_step,batch_size,output_dim
        Total_Acc = Total_Acc + len(Real)        
        Cross = list(set(Predicted).intersection(set(Real)))
        if len(Cross) == 0:
            print(11)
            continue
        #print(Cross)
        Pred_Acc = Pred_Acc + len(Cross)
        Acc_single = len(Cross)/len(Real)
        print(Acc_single)
        Acc_Real[seq_step] = Acc_Real[seq_step] + len(Real)
        Acc_Iden[seq_step] = Acc_Iden[seq_step] + len(Cross)
        # MSE/MAPE
        F_error = np.mean((final_preds_F[0,seq_step,batch,:]-Test_Output_Risk_B[batch,seq_step,:])**2)
        C_error = np.mean((np.int32(final_preds_C[0,seq_step,batch,:])-Test_Output_Ycoarse_B[batch,seq_step,:])**2)
        F_mape = np.mean(abs((final_preds_F[0,seq_step,batch,:]-Test_Output_Risk_B[batch,seq_step,:])/Test_Output_Risk_B[batch,seq_step,:]))
        Test_Output_Ycoarse_B1 = np.where(Test_Output_Ycoarse_B==0,-1,Test_Output_Ycoarse_B)
        final_preds_C1 = np.where(final_preds_C==0,-1,final_preds_C)
        C_mape = np.mean(abs((np.int32(final_preds_C1[0,seq_step,batch,:])-Test_Output_Ycoarse_B1[batch,seq_step,:])/Test_Output_Ycoarse_B1[batch,seq_step,:]))
        FError = FError + F_error
        CError = CError + C_error
        FMAPE = FMAPE + F_mape
        CMAPE = CMAPE + C_mape
FError = FError/(Batchsize*Output_seq_len)
CError = CError/(Batchsize*Output_seq_len)
FMAPE = FMAPE/(Batchsize*Output_seq_len)
CMAPE = CMAPE/(Batchsize*Output_seq_len)
print("Acc:",Total_Acc,Pred_Acc,Pred_Acc/Total_Acc)
print("MSE/(F/C):",FError,CError,"MAPE(F/C)",FMAPE,CMAPE)


# Test process
# F_error = np.mean((final_preds_F[0,seq_step,batch,:]-Test_Output_Risk_B[batch,seq_step,:])**2)
# C_error = np.mean((np.int32(final_preds_C[0,seq_step,batch,:])-Test_Output_Ycoarse_B[batch,seq_step,:])**2)
# F_mape = np.mean(abs((final_preds_F[0,seq_step,batch,:]-Test_Output_Risk_B[batch,seq_step,:])/Test_Output_Risk_B[batch,seq_step,:]))
# Test_Output_Ycoarse_B1 = np.where(Test_Output_Ycoarse_B==0,-1,Test_Output_Ycoarse_B)
# final_preds_C1 = np.where(final_preds_C==0,-1,final_preds_C)
# C_mape = np.mean(abs((np.int32(final_preds_C1[0,seq_step,batch,:])-Test_Output_Ycoarse_B1[batch,seq_step,:])/Test_Output_Ycoarse_B1[batch,seq_step,:]))



In [ ]:
# 其他可能用到的代码（Other codes that can be utilized for deriving results）

In [ ]:
import numpy as np
F2C=np.load('Dataset/Cluster_18_6.npy') 

def Fine2Coarse(FineMap):
    Coarse_Y = np.zeros(18,)
    for i in range(18):
        for k in range(36):
            if(F2C[i,k]!=0):
                Coarse_Y[i] = Coarse_Y[i] + FineMap[F2C[i,k]] 
    return Coarse_Y
#FineMap1 = np.ones([354,1])
Coarse_Y = Fine2Coarse(FineMap1)

In [ ]:
# 分时段评估
K = 30
Total_Acc = 0
Pred_Acc = 0
Acc_Iden = np.zeros([6])
Acc_Real = np.zeros([6])
for batch in range(Batchsize):
    for seq_step in range(Output_seq_len):
        #batch = 5
        #seq_step = 0
        Real = list(np.flatnonzero((Test_Output_Risk_B[batch,seq_step,:])>0)) # 10,6,354 batch_size,time_step,output_dim
        Predicted = Risk_Proposal(final_preds_F[0,seq_step,batch,:],final_preds_C[0,seq_step,batch,:])
        # Predicted = list(np.argsort(-final_preds_F[0,seq_step,batch,:])[0:K]) # 1,6,10,354  1,time_step,batch_size,output_dim
        Total_Acc = Total_Acc + len(Real)        
        Cross = list(set(Predicted).intersection(set(Real)))
        print(Cross)
        Pred_Acc = Pred_Acc + len(Cross)
        Acc_Real[seq_step] = Acc_Real[seq_step] + len(Real)
        Acc_Iden[seq_step] = Acc_Iden[seq_step] + len(Cross)
print(Total_Acc,Pred_Acc,Pred_Acc/Total_Acc)

In [ ]:
# 进行提名 返回哪些是高风险的区域
Cluster_results = np.load('Dataset/Cluster_18_6.npy')

def Risk_Proposal(Task1,Task3):
    count1=0
    count2=0
    Count=0
    Dim_o3 = 18
    A_risk=np.zeros([Cluster_results.shape[0],Cluster_results.shape[1]])
    for i in range(Dim_o3):
        for j in range(np.flatnonzero(Cluster_results[i]).shape[0]):
            if Cluster_results[i,j]!=0:
                #print(Cluster_results4[i,j])
                A_risk[i,j]=Task1[Cluster_results[i,j]]

    risk_index=[]    
    for i in range(Dim_o3):
        k=np.int16(Task3[i])
        #print("k:",k)
        if i!=0:
            Sort1=np.array(-A_risk[i,0:np.flatnonzero(Cluster_results[i]).shape[0]+1]).argsort()
        else: 
            Sort1=np.array(-A_risk[i,1:np.flatnonzero(Cluster_results[i]).shape[0]+1]).argsort()
       # print(Sort1)
        if (k>=2):
            #print(i,Sort1)
            SS=Sort1[0:k]
            risk_index.append(list(SS))
        if (k<=1): #只要k小于等于1则用这个方法
            risk_index.append([Sort1[0],Sort1[1]])
#             if np.flatnonzero(Cluster_results[i]).shape[0]>13:
#                 risk_index.append([Sort1[0],Sort1[1]])
#             else:
#                 risk_index.append([-1])
    #print("index",risk_index)
    #由cluster的索引号映射回原来的ID
    Highrisk_ID=[]
    for i in range(Dim_o3):
        #print("risk_index[i]:",risk_index[i])
        if  len(risk_index[i])>2:
            for kk in risk_index[i]:
                #print("kk:",kk)
                Highrisk_ID.append(Cluster_results[i,np.int(kk)])
                #print(1,i,np.int(kk))
        if len(risk_index[i])<=1:
            if risk_index[i][0]!=-1:
                Highrisk_ID.append(Cluster_results[i,int(risk_index[i][0])])
                #print(Cluster_results[i,int(risk_index[i][0])])
    Highrisk_ID = np.unique(Highrisk_ID)    
    return list(Highrisk_ID)